Uses the prepared [melspectrograms](https://librosa.github.io/librosa/generated/librosa.feature.melspectrogram.html) and [Mel-frequency cepstral coefficients (MFCCs)](https://librosa.github.io/librosa/generated/librosa.feature.mfcc.html) to train and evaluate a CNN-based classifier.

This version includes MFCCs in the feature space. Instead of adding this array as a separate input, or stacking it on top, it is simply overwrites the bottom 20 frequency bands of the spectrogram. Since they share the same `time` axis, this should be a good approach. This is also a convenient means of filtering out the lower frequencies.

This version also include Dropout(rate=0.2) after each MaxPooling layer. 

This version does **not** filter out quiet frames.

In [ ]:
import pandas as pd
import numpy as np
import librosa as lr
from librosa.display import specshow
from glob import glob
import os
from IPython.display import Audio
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix
import keras
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

## Data Loading

In [ ]:
def get_full_path(sample): return os.path.join(sounds_dir, sample['file_name'])
sounds_dir = "../input/xenocanto-avian-vocalizations-canv-usa/xeno-canto-ca-nv/"
# sounds_dir = "../input/xeno-canto-ca-nv/"

df = pd.read_csv("../input/xenocanto-avian-vocalizations-canv-usa/xeno-canto_ca-nv_index.csv" ).drop('Unnamed: 0',axis=1)
display(df.head(2))
# df = pd.read_csv("../input/xeno-canto_ca-nv_index.csv")
files_list = glob(os.path.join(sounds_dir,"*.mp3"))
print("%i mp3 files in %s"%(len(files_list), sounds_dir))
print("%i samples in index."%len(df))

melspec_dir = "../input/avian-vocalizations-melspectrograms-log-norm/"
melspec_features_dir = melspec_dir + "/melspectrograms_logscaled_normalized/features"

# shapes_df = pd.read_csv(melspec_dir+"/feature_shapes.csv")
shapes_df = pd.read_csv("../input/avian-vocalizations-spectrograms-and-mfccs/feature_shapes.csv" ).drop('Unnamed: 0',axis=1)
shapes_df.head(2)

In [ ]:
import re
def parse_shape(shape_str):
    """Shape was saved in feature_shapes as a string. Woops. Parse out the values. """
    a,b = re.search('\((\d+), (\d+)\)',shape_str).groups()
    return int(a), int(b)

def log_clipped(a):
    """Convenience function to clip the input to positive values then return the log.""" 
    return np.log(np.clip(a,.0000001,a.max()))

## Train/Test Split

In [ ]:
y_english_labels_entire_dataset = [s['english_cname'] for i,s in df.iterrows()]
label_encoder = LabelEncoder().fit(y_english_labels_entire_dataset)
y_encoded_entire_dataset = np.array(label_encoder.transform(y_english_labels_entire_dataset))

n_classes = len(label_encoder.classes_)

train_df = pd.read_csv("../input/avian-vocalizations-partitioned-data/train_file_ids.csv",index_col=0)
display("Training data:",train_df.head(2))
test_df = pd.read_csv("../input/avian-vocalizations-partitioned-data/test_file_ids.csv",index_col=0)
display("Test data:",test_df.head(2))
# X_train, X_test, y_train, y_test = train_test_split(
#     np.array([s['file_id'] for i,s in df.iterrows()]), 
#     y_encoded_entire_dataset, 
#     test_size=1/3, 
#     stratify=y_encoded_entire_dataset, 
#     shuffle=True, 
#     random_state=37,
# )
X_train = list(train_df.index)
y_train = list(train_df.label)
print("Training data len:",len(X_train), len(y_train))
X_test = list(test_df.index)
y_test = list(test_df.label)
print("Test data len:    ",len(X_test), len(y_test))

## Data Generator
Create a Data Generator to generate fixed-length samples from random windows within clips

In [ ]:
from sklearn.preprocessing import StandardScaler
mfcc_scaler = StandardScaler()
for file_id in shapes_df.file_id:
    mfcc = np.memmap('../input/avian-vocalizations-spectrograms-and-mfccs/mfccs/features/XC%s_mfcc.dat'%file_id, 
        shape=parse_shape(shapes_df[shapes_df.file_id==file_id]['mfcc_shapes'].values[0]),  dtype='float32', mode='readonly')
    mfcc_scaler.partial_fit(mfcc.flatten().reshape(-1, 1))
print("MFCC scaler:",mfcc_scaler.mean_, mfcc_scaler.var_, np.sqrt(mfcc_scaler.var_))

In [ ]:
import keras
from keras.utils import to_categorical

# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class AudioFeatureGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size, n_frames=128, n_channels=1,
                 n_classes=10, shuffle=False, seed=37):
        'Initialization'
        self.n_frames = n_frames
        self.dim = (128+20, self.n_frames)
        self.batch_size = batch_size
        self.labels = {list_IDs[i]:l for i,l in enumerate(labels)}
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.seed = seed
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.seed)
            self.seed = self.seed+1 # increment the seed so we get a different batch.
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
#         X = np.empty((self.batch_size, *self.dim, self.n_channels))
        X = np.empty((self.batch_size, 128+20, self.dim[1], self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int) # one-hot encoded labels

        for i, ID in enumerate(list_IDs_temp):
            sg_lognorm = np.memmap(os.path.join(melspec_features_dir,'XC%s_melspectrogram_logscaled_normalized.dat'%ID), 
                    shape=parse_shape(shapes_df[shapes_df.file_id==ID]['melspectrogram_shapes'].values[0]),  dtype='float32', mode='readonly')
#             sg = np.memmap('../input/avian-vocalizations-spectrograms-and-mfccs/melspectrograms/features/XC%s_melspectrogram.dat'%file_id, 
#                     shape=parse_shape(shapes_df[shapes_df.file_id==file_id]['melspectrogram_shapes'].values[0]),  dtype='float32', mode='readonly')
            mfcc = np.memmap('../input/avian-vocalizations-spectrograms-and-mfccs/mfccs/features/XC%s_mfcc.dat'%ID, 
                    shape=parse_shape(shapes_df[shapes_df.file_id==ID]['mfcc_shapes'].values[0]),  dtype='float32', mode='readonly')
            # Normalize MFCCs
            mfcc = mfcc_scaler.transform(mfcc)
            
            # Filter out quiet frames, thanks to https://www.kaggle.com/fleanend/extract-features-with-librosa-predict-with-nb
            # Take mean amplitude M from frame with highest energy
#             m = sg[:,np.argmax(sg.mean(axis=0))].mean()
#             # Filter out all frames with energy less than 5% of M
#             mask = sg.mean(axis=0)>=m/20
#             sg = sg[:,mask]
#             sg_lognorm = sg_lognorm[:,mask]
#             mfcc = mfcc[:,mask]
            
            d_len = mfcc.shape[1] - self.dim[1]
            np.random.seed(i+self.seed) # For reproducible results
            if d_len<0: # Clip is shorter than window, so pad with mean value.
                n = int(np.random.uniform(0, -d_len))
                pad_range = (n, -d_len-n) # pad with n values on the left, clip_length - n values on the right 
#                 sg_cropped = np.pad(sg, ((0,0), pad_range), 'constant', constant_values=sg.mean())
                sg_lognorm_cropped = np.pad(sg_lognorm, ((0,0), pad_range), 'constant', constant_values=0)
                mfcc_cropped = np.pad(mfcc, ((0,0), pad_range), 'constant', constant_values=0)
            else: # Clip is longer than window, so slice it up
                n = int(np.random.uniform(0, d_len))
#                 sg_cropped = sg[:, n:(n+self.dim[1])]
                sg_lognorm_cropped = sg_lognorm[:, n:(n+self.dim[1])]
                mfcc_cropped = mfcc[:, n:(n+self.dim[1])]
                
            X[i,] = np.concatenate([sg_lognorm_cropped.reshape(1,128,self.dim[1],1), mfcc_cropped.reshape(1,20,self.dim[1],1)], axis=1)
#             X[i,] = sg_lognorm_cropped.reshape(1,128,self.dim[1],1)
            # Overwrite the bottom of X with MFCCs (we don't need the low frequency bands anyway) 
#             X[i,:20] = mfcc_cropped.reshape(1,20,self.dim[1],1)
            y[i,] = to_categorical(self.labels[ID], num_classes=self.n_classes)

        return X, y

## Sample Visualizations

In [ ]:
from itertools import islice
generator = AudioFeatureGenerator(X_test, y_test, batch_size=1, shuffle=True, seed=37, n_frames=128, n_classes=n_classes)
for g in islice(generator,0,4): # show a few examples
    for i,spec in enumerate(g[0]): 
        plt.figure(figsize=(10,4))
        spec_ax = specshow(spec.squeeze(), x_axis='time', y_axis='mel')
        plt.title(label_encoder.classes_[np.argmax(g[1][i])])
        plt.colorbar()
        plt.show()

In [ ]:
def vis_learning_curve(learning):
    train_loss = learning.history['loss']
    train_acc = learning.history['acc']
    val_loss = learning.history['val_loss']
    val_acc = learning.history['val_acc']

    fig, axes = plt.subplots(1, 2, figsize=(20,4), subplot_kw={'xlabel':'epoch'} )
    axes[0].set_title("Accuracy")
    axes[0].plot(train_acc)
    axes[0].plot(val_acc)
    axes[0].legend(['training','validation'])
    axes[1].set_title("Loss")
    axes[1].plot(train_loss)
    axes[1].plot(val_loss)
    axes[1].legend(['training','validation'])

    best_training_epoc = val_loss.index(np.min(val_loss))
    axes[0].axvline(x=best_training_epoc, color='red')
    axes[1].axvline(x=best_training_epoc, color='red')


## Model Training

In [ ]:
print("Accuracy by random guess: %.4f"%(1/len(df['english_cname'].unique())))

In [ ]:
n_splits = 3
n_epochs = 150
sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=1/4, random_state=37)
scores = []
params = {#'dim': (128,256),
          'n_frames': 128,
          'n_classes': n_classes,
          'n_channels': 1}
for cv_train_index, cv_val_index in sss.split(X_train, y_train):
    training_generator = AudioFeatureGenerator([X_train[i] for i in cv_train_index], [y_train[i] for i in cv_train_index], 
                                               batch_size=64, shuffle=True, seed=37, **params)
    validation_generator = AudioFeatureGenerator([X_train[i] for i in cv_val_index], [y_train[i] for i in cv_val_index], 
                                               batch_size=len(cv_val_index), **params)
    dim = training_generator.dim
    
    partial_filename = "cnn.16.32.64.split%02i"%len(scores)
    checkpointer = ModelCheckpoint(filepath='weights.best.%s.hdf5'%partial_filename, verbose=1, save_best_only=True)
    
    # From My Udacity Dog Project Image Classifier
    model = Sequential()
    model.add(Conv2D(64,3,input_shape=(dim[0], dim[1], 1),padding='valid',activation="relu"))
    model.add(MaxPooling2D(pool_size=3))
    model.add(Dropout(rate=.2))
    model.add(Conv2D(64,3,padding='valid',activation="relu"))
    model.add(MaxPooling2D(pool_size=3))
    model.add(Dropout(rate=.2))
    model.add(Conv2D(64,3,padding='valid',activation="relu"))
    model.add(MaxPooling2D(pool_size=3))
    model.add(Dropout(rate=.2))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(n_classes, activation="softmax"))
    model.summary()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    learning = model.fit_generator(
                training_generator, 
                validation_data=validation_generator,
                epochs=n_epochs, 
                steps_per_epoch=np.ceil(len(cv_train_index)/training_generator.batch_size),
                validation_steps=1,
                callbacks=[checkpointer], 
#                 use_multiprocessing=True, workers=4,
                verbose=0, )
    pd.DataFrame(learning.history).to_csv('training_history_split%i.csv'%len(scores), index_label='epoch')
    vis_learning_curve(learning)
    plt.savefig("learning_curve.%s.png"%partial_filename)
    plt.show()
    acc_at_min_loss = learning.history['val_acc'][np.argmin(learning.history['val_loss'])]
    scores.append(acc_at_min_loss)
    print("Split %i: min loss: %.5f, accuracy at min loss: %.5f"%(
        len(scores), np.min(learning.history['val_loss']), acc_at_min_loss ))
print("Cross Validation Accuracy: mean(val_acc[argmin(val_loss)]): %.4f"%(np.mean(scores)))